open image in deepzoom (DeepPATH) in cmd

include/code/openslide-python/examples/deepzoom/deepzoom_server.py include/image/raw/S19005645 A1-2.svs

a.svs = S19005645 A1-2.svs
b.svs = S19009823 A1-2.svs

tile slide to 40x 256*256
AFB_5081A.svs

python3 include/code/DeepPATH/DeepPATH_code/00_preprocessing/0b_tileLoop_deepzoom4.py -s 299 -e 0 -j 32 -B 25 -m 40 -o include/image/tile "include/image/raw/aaa.svs"

In [1]:
# classify tile image into positive and negative
import shutil, os
path = "/home/napasiri/python/AFB/include/image/data_extraction/red.jpg"
shutil.move(path, "/home/napasiri/python/AFB/include/image")

'/home/napasiri/python/AFB/include/image/red.jpg'

In [2]:
from skimage import measure
import cv2
import shutil, os
import numpy as np

des_neg = "/home/napasiri/python/AFB/include/image/negative"
des_pos = "/home/napasiri/python/AFB/include/image/positive"

def classifier(img, hsv_lower_limit = [110,30,164], hsv_upper_limit = [138,54,195]):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_mark_H = cv2.inRange(img_hsv, np.array(hsv_lower_limit), np.array(hsv_upper_limit))
    img_dilate = cv2.dilate(img_mark_H, None, iterations = 1)
    
    labels = measure.label(img_dilate, neighbors=None, background=0)
    a = 0
    
    for label in np.unique(labels):
        if label==0:
            continue
        labelMask = np.zeros(img_mark_H.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPix = cv2.countNonZero(labelMask)

        if numPix > 5:
            a +=1
        else:
            pass
        
    if a==0:
        shutil.move(path, des_neg)
    elif a>0:
        shutil.move(path, des_pos)
    else:
        print("ERROR")
    

In [ ]:
path_main = "/home/napasiri/python/AFB/include/image/tile"
path_main_lst = os.listdir(path_main)
for i in path_main_lst:
    b = os.listdir(path_main + "/" + i)
    path1 = path_main + "/" + i
    for i in b:
        path2 = path1 + "/" + i
        c = os.listdir(path2)
        for i in c:
            path = path2 + "/" + i
            img = cv2.imread(path)
            classifier(img)

In [ ]:
for i in os.listdir("/home/napasiri/python/AFB/include/image/test_img"):
    path_test = "/home/napasiri/python/AFB/include/image/test_img/" + i
    img_test = cv2.imread(path_test)
    classifier(img_test)


coarsely separate positive and negative 64*64 tile

In [2]:
import shutil, os
import cv2
import numpy as np

In [16]:
path = '../image/tile/a_64/40.0/'
tile_data = []
for i in os.listdir('../image/tile/a_64/40.0/'):
    img_data = []
    B = []
    G = []
    R = []
    img_data.append(i)
    img = cv2.imread(path + i)
    img = np.asarray(img)
    for row in img:
        for pixel in row:
            B.append(pixel[0])
            G.append(pixel[1])
            R.append(pixel[2])
    B = sum(B)/len(B)
    G = sum(G)/len(G)
    R = sum(R)/len(R)
    img_data.append(B)
    img_data.append(G)
    img_data.append(R)
    tile_data.append(img_data)

# get tile_data = [[img_name, average B, G, R], ...]

In [17]:
len(tile_data)

25661

In [18]:
print(tile_data[:10])

[['308_35.jpeg', 222.5380859375, 186.83349609375, 137.35498046875], ['73_273.jpeg', 223.72412109375, 186.28857421875, 134.13720703125], ['42_279.jpeg', 229.779052734375, 207.7724609375, 162.273193359375], ['113_304.jpeg', 227.00732421875, 200.5185546875, 156.888671875], ['214_96.jpeg', 227.269775390625, 201.08642578125, 158.3173828125], ['88_269.jpeg', 226.331298828125, 194.00732421875, 151.650390625], ['276_151.jpeg', 226.04248046875, 198.067626953125, 157.590576171875], ['131_106.jpeg', 221.54931640625, 185.822998046875, 130.715576171875], ['256_192.jpeg', 228.822998046875, 201.318115234375, 163.24462890625], ['255_148.jpeg', 225.64404296875, 195.795166015625, 148.473388671875]]


In [ ]:
for i in range(len(tile_data)):
    if tile_data[i][R] >= 